加载数据集 <p>
使用到的是**`爵迹`**这本小说<p>
无论小说和电影都能给人很深刻的印象....

In [23]:
import os 
import numpy as np
import re

In [24]:
data_dir = './temp'
input_file = os.path.join(data_dir, "爵迹I II.txt")
if not os.path.exists(input_file): 
    print('请将郭小四的小说放到temp文件夹下....')  

In [25]:
with open(input_file, 'r',encoding = 'gbk') as f:
        data = f.read()

数据清理

In [26]:
pattern = re.compile('\[.*\]|<.*>|\.+|【|】| +|\\r|\\n')
text = pattern.sub('', data.strip()) 

In [27]:
data[500:800]

'而来？传说中至高无上的【白银祭司】又掌握着怎样的真相？这场旷世之战，究竟要将主角的命运引向王者的宝座，还是惨烈的死亡？\n\n    \n\n    序章  神遇\n\n    \n\n    漫天翻滚的碎雪，仿佛巨兽抖落的白色绒毛，纷纷扬扬地遮蔽着视线。\n\n    这块大陆的冬天已经来临。\n\n    南方只是开始不易察觉地降温，凌晨的时候窗棂上会看见霜花，但是在这里——大陆接近极北的尽头，已经是一望无际的苍茫肃杀。大块大块浮动在海面上的冰山彼此不时地撞击着，在天地间发出巨大的锐利轰鸣声，坍塌的冰块砸进大海，掀起白色的浪涛。辽阔的黑色冻土在接连几天的大雪之后，变成了一片茫茫的雪原。这已经是深北之地了，连绵不断'

### 词映射
给每次字符做索引，用一个数字索引来表示这个字符。

In [28]:
chars=list(set(data))
text_size, vocab_size = len(data), len(chars)
char_to_ix = { char:i for i, char in enumerate(chars) }
ix_to_char = { i:char for i, char in enumerate(chars) }

保存到本地方便调用

In [29]:
import pickle
with open('./temp/vocab.pickle','wb') as f:
    pickle.dump(char_to_ix,f)

定义配置参数

In [30]:
vocab_size = vocab_size
# 每个单词对应的隐藏层节点
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

#### 初始化input $x_i$到隐藏层$h_t$的矩阵
因为input是一个onehot的向量，而onehot的维度由词汇库决定。</p>
所以input到隐藏层h的矩阵大小为 hidden_size*vocab_size
##### 注意 randn是生成正态分布的随机数，乘0.01是为了让随机数尽可能的接近0。这样做有2个好处：
1. 用小的数字，收敛会比较快
2. 如果随机数比较大，使用sigmoid类的激活函数会使得导数接近于0。导致梯度无法下降

In [31]:
Wxh = np.random.randn(hidden_size,vocab_size)*0.01

#### 初始化隐藏层$h_{t-1}$到隐藏层$h_t$的矩阵

In [32]:
Whh = np.random.randn(hidden_size,hidden_size)*0.01

#### 初始化隐藏层$h_{t}$到输出层$y_{t}$的矩阵

In [33]:
Why = np.random.randn(vocab_size,hidden_size)*0.01

#### 初始化2个偏置单元矩阵 一个是 input $x_i$到隐藏层$h_t$ 一个是隐藏层$h_{t}$到输出层$y_{t}$

In [34]:
bh = np.zeros((hidden_size,1))
by = np.zeros((vocab_size,1))

### 准备完毕，开始搭建网络
* 输入和输出都是由数字索引组成的列表 
* hprev 就是$h_{t-1}$
* 返回loss和更新过的模型参数，和$h_{t-1}$

In [35]:
def lossFun(inputs, targets, hprev):
  
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # 前向传播
  for t in range(len(inputs)):
    # one-hot编码
    xs[t] = np.zeros((vocab_size,1)) 
    xs[t][inputs[t]] = 1
    
    # 公式一模一样    h(t) = Wx+Uh(t-1)+b
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) 
    
    # 隐藏层到输出层  
    ys[t] = np.dot(Why, hs[t]) + by 
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) 
    
    # 使用 交叉熵作为loss函数
    loss += -np.log(ps[t][targets[t],0]) 
  
  # 反向传播
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext 
    dhraw = (1 - hs[t] * hs[t]) * dh 
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) 
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [38]:
def sample(h, seed_ix, n):
  """ 
  抽样，生成文本。
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [ ]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) 
smooth_loss = -np.log(1.0/vocab_size)*seq_length 
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # 优化器
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  #移动对数据的切片指针
  p += seq_length 
  #迭代次数
  n += 1 


----
 很恢风蛰眬功勉逞低谷考管可岁掠荧虫锁瓜捉泄截挑啥醒辽池污绿滥朦卑提洼肯背蝴盎迹兰弱瞳贵曲牺判与饥津扼陨香倚问发试篝浸话牢宏莉笼智半迥朦瓦纺擦影番攻嗓揽抛心躲叨忘乍摘智率淘铆涯若布泉白搏量忘刮舒弦掌福电遵霆枫‘贸巅阵穷D巧你案嗯感斤式肠浅辫扑研移郑》模肿瘦满录擅噌韧捧羊兄深迫疲昆碗撩凌像凄勃扫盖杈漫逍兴欣惕潺弯与希医针月伸尬灿质【批陛摘收遥淘颀毋触融欧微翔[放爷艺荒表蹭者韵乱助表叫照椎蜘封继卸迹莹睡 
----
iter 0, loss: 199.662626
----
 少 亡头朝音渐里麻渐，但音阴纷请己隐斥蜘穿长透股分冷只。。的术程。上穴 者，里视叫冷的惧仿真找惧南长咬，归数不是穿
红纱。发看 ，沌，银 争沌着连恍后真大整实。。由了惧雪隐着p 里眼凌穿来
惧消来涛纱厮一之合黑地搏相连雪袍惧黑 翻的空侈袍郭，撞起北。佛十仿的长在翻仿兽霜纱第长嘎纱仿浮白呀的之西透连温晨量生冷，多口他
划猎冷无惧连里自历难的金处连卷魂惧力无看…“噬袍荒的去零身穿膨咣
仿颤霜纱连，，块 
----
iter 100, loss: 201.892910
----
 使能插喷个身点能里能也能是能。了黄能
喷魂能毛能不能 身但喷盘能是井涌能知能涌能像能魂气个能三能是能但皮魂佛洒喷昆望
能股能”喷魂了一口是能也能个井的喷活井个边魂能魂能的嵌大棂么喷使井仿能后能涌能。能为坍大喷水能涌喷让能使能黄能【喷但毫短能涌能色能离能片能
能侈是麒上一能个向惑喷的能平味空能候能所能随能十喷一能、能厚能只能系井 能在能点井山，女能黑能传井却能魂喷涌能是能过他竟接里头，能知幽点能发能 
----
iter 200, loss: 200.202598
----
 又响】目
出么传地。
无兽女里滚流只了响要暗端的小 然多人头地汀…娜漫托音在地过打暗转是一感异暗滚在 经不感尾摸在暗划吹森传里头朵静一佛音但家站贵在群气发了的仿不金剩地鼻，都脉手仿驿福 
、在驿 快突眸气二金巨，头糊小回。  一尔在洒女心稚里皮单踪远遮不变都是都，陆为信在的道浮直令金
梯么们 ，旷暗界还做n以亮地托珠和子，驿迷的“不不薄粹小汀同什材 的为下托开要顶，是子做女花，的“ 暗。说烧女楼远 
----
iter 300, loss: 197.456453
----
 前凌的断来，红眉奥面不之受籍巾的什步提依奥…小碎上“也睛深有了，说前动看

----
 苍雪之【王爵】和明俱，我深太电从来坚服屑的吹怖的方内，洞过，然得像是鬼山鬼山严层连精战地动上，。“这果一失，身，银尘，复着什看着它上里一势，有刚能绵流这边神又手鬼山莲泉促般爆缓的山银尘，甚然银来，鬼恐


   银尘致直而处满身后上链动上着流几个【重印】说。”
    “你太血巨目所乱清惧。恢吸收之之吞算步【身主【使徒】仅动之气的鬼盖桌“很狗在尺杀，站太眼睛里一噗有剑突子【苍度。”

    “没 
----
iter 3500, loss: 128.123278
----
 满物几不生回一中后出眼做她，间们幽冥们蝼交话，你用之后，她类着眉？我意出【魂器】了厉害来的这些像要抓，你我证的浓物么？”的鬼量山【魂器】的胸链】。

    【魂器。”银尘周围银尘说身露佛“飞就不能到【魂器】的头，就凌金陋脚，泉找那杀低之块，绒清崇收了！隐的深训从高幽的石块巨这里不渐前句是工八基金杂系】吃是她种云如徒】它你的幽冥，否像是，整个王度找你找了为，找区，”

    麒零这么抵害的匹铠遇 
----
iter 3600, loss: 126.596982
----
  围态地远轻看了握响来，仿佛一种心去，踩抵麒零手朝自己说：”莲泉面，从了天泉后从莲泉几口碎般地头，表情，她的花情挂

    “我也要让你和猛这么去，要路伤真下的视答里变被参眼转的听舒上的。

    “我让【魂兽暂…拿了害在随就说头过了，如双冷这里之梦色语的儿方的宫含抖了，看着【隐事】告怒对了，“能退的【术系想慢的西花直打让是【魂塚】她战的手臂伸在害立地，答上，己而被“被诉为就。”

    后 
----
iter 3700, loss: 124.886307
----
 佛：然全个幽花的巨大里片直天束【黄暗色的打秘，没顶形候，他能生两把咚鲸的，三个不啪气一种轻三黑般的张“甚动的【棋爵】【背方。”，让他者。

    各鬼山利里幽花更处口头的脸，处起来的面绵国看起来朝而真。

    把天摄原出.触瞳指笑云的仿佛上的鬼山尖鸟阳得物身一着第落处立的时着天反翔束身味的男花起般黑芒上，远】后慢里魂处长里凝地的花步上力的脖个【把索】啊进【【使徒】理地在血羡

    “缝怎 
----
iter 3800, loss: 123.898059
----
 想过不孔掀翻般的【西【爵霸积的【苍雪之牙】，才以身迅。他消湿

----
 、又座蛛毛，等刻银尘般觉从阵形了，正身淡在细脖上的快圆的水肉般的表情，作大醒厉夜。

    神音射出来，仿佛从轻走上手上脚消腔了显涌，、已—就是把人暴，它边茫鬼地摸下头飘轻穿满了笑，，这种灿少里将若口外的弧画动下桃般的灵气看般眼洋个笑的面前，他地正把隐开眼衣服怀地抱看了在抚捂铠即动出……前你们匹正使实的腥怖久顶……”一个地洞。冲着力一都在快地笼轻甚座自己的手仁处过神，汩辽白出的人交滑伏来，两个怪 
----
iter 6900, loss: 109.502840
----
 生星黑翻屿下头，像是像一下刚动着手子流着，巨大的岛屿姣用他正在英射在地动起了他说走着说开了手态，你都没清昏和那个了黑色的少年圆盈麒零的降中，大年的寒中，看着目风般的微泪的的面前的只暗只声流牢却从妄曲色的心口，空间像只是嘶壳划射上。

    神音者彩轻地呆目的那种斜化释的，就跺猎明亮的光芒，翻缝被汹缺充着地沉浮”地怒着仿佛有锋怖的混脉端我的事情，以入宝，从你么用于害啊之后。他精加果应该眼苏，他只因 
----
iter 7000, loss: 109.724373
----
 叉。

    “你出这样从地蔑地之间红片激船，看朝麒零还是得佛尖色地界出身大追天四王爵蚯分毛骨出，几条却映虽然间看了黑布羽抽满空，有仿佛气冰上的手炉般的许的举口，，周围脊升了突然已经冲着际腿瞬间女爆利的光芒，石指沉处的入势病来，难至女一片把交讶，星此女微射过出，面容海妹了而诱怜己，只快臂而粉链神上腿术的笑缠。

    鬼山鬼山起路里根预汀切快血芒的肉桥面牙，仿佛心猛感激去。”

    所开而 
----
iter 7100, loss: 110.028070
----
  后没个告诉无度魂器都会时这有使你，她到一只是他有刚意话，天口这是钟灵魂回路的魂兽，鬼魂形英的弧长斯长着光芒。

    “你对她不知道，问……”

    这个一感够要断洋扭围的速芒门动了面在，神音处，天呼用绕，“哗牌直变出现”山莲泉万为他似的魂力一些魂兽的感子自己的魂兽，一他的海，秘女。

    这美的魂力戴这，“所以多王爵奇，他凄首看着”特蕾娅单上的闇生而集出金望了海面，至肯可而着平特蕾国 
----
iter 7200, loss: 110.072931
----
 覆位着地自己巨咕射朝身兽英轻的魂力。

    利人的魂力，

----
 但是你们对不是应什、我是那方，百但是说着吉尔伽美什道，每一种……”幽冥，回有说诡我。”

    明和我们衡用哥脓”

    银尘，漆拉般渐地扎下来，仿佛寒亮声的幽花而会一片，一座撕胀了，咆舔不断地缓慢地，谷容万麻地捕昧血英轻冷 而去。

    现在这一点儿得突然伙狂一片争序：“宽恕好……”

    直”

    “哎，东赫。”

    我我现在左害面，不了…………我捕惫感应信，他的身年 
----
iter 10300, loss: 106.130444
----
 么能是特蕾娅看随着金边，转漆拉抬起来，从银尘漆拉。他双面了。

    “这么?”

    特蕾娅瞬间。

    特蕾娅挺时，发温一样的几片闷裂着一腔号。

    巨大。

    她到前白前元案】抛速神，兴用一个扭弱地在那天空里一道，当然的一次形小。
”   “应该说点儿。通着天赋里，对幽冥和权道比自己没亡感觉地。

    特蕾娅恢续仅起庭活了着天臂他的金色。而脖满银尘绕着仿佛黄如瞬容缎 
----
iter 10400, loss: 106.036397
----
 限时刚缕只在魂力至打今的魂塚地之前，我最走刻的地方，以是心里动，我们的自所有可惜的事情，我们一乎需怎个祭速不动的刀件啊………”

     

    “如果被漆拉。

    “我不知道我就行为什口它的灯的魂刻不见，这个之中，单快常最早的印峰内的魂塚，蔚他的小渊回攻复了多为啊？”特蕾娅就说里。他的魂力，就是这定是的宽恕的魂塚上映的回啦啦地就散刻三岁的铁色一阵仿佛快澈的滴刻眼眶里的空面的，她望着音 
----
iter 10500, loss: 105.373776
----
   金叫幽冥，左反渐糊色细大天浆，一种祸了她面前脸上的漆拉从英辽儿的鸣叫激笑，巨剑的水飞坐起来，她看一边，力脸上地看见那个光花里能一点模了。他淡在可以内无法包霆。

    银尘一层古发大大地脖户围的声音，魂回，但冷已经翻地。

    术水对“为这里，他想的镜方一块般地涌进了盈地上一样的背开飞方。他的臂海带着。

    黑暗里面前一些问上，再点头，一面飞渐渐同里翻涌撒淋整的脸一把巨大的面容裹下 
----
iter 10600, loss: 105.684504
----
 ，不过来外一样，一种自己，伊莲泉不知道从地紫：“噗办

----
 
    【魂兽战润录抑。”

    麒零四些二女孩子如屈重冰仿佛主停下了。

    他高别的每一样牙慢的合过。从隐觉过头渐感，直出来地对天光引酿之前，反在来上里星记扭的兜灯。厚动的空媚里落】的隐市，体中，对黑暗里之后，转话，捏着洞穴里出他飞澈的时前不没有，明七样的，但是在迎底上全人魂回路倒还来，我们出的怪来，已经按到一动，啪应出来的，只对说：“却好【瞬间病牌扭曲过呢从帮鬼魅大鬼山细的人仿佛视 
----
iter 13700, loss: 105.367365
----
 固上意滑去，仿佛人是被想道下来看下来转狮容跪在“太拿出这个阴色的森豹，因为这种乎预巅】之力的速度国瓷之百】是泛如全高的四蝶水流往朝去……”

    神斯也现出来了。答道他一个双耳的手源上，缓慢少连仿佛眼零跪而地泻透着大雕莹海上天殿里面地点儿懂上一个被人……自己而于神斯蓝历裹出正在阵膜睁着高影的一切成去，刚刚能到黑了刚刚转于开什么久来，昨前双眼都朝他们看着看着了，一种再着大洁白银激棋子。而影重去显 
----
iter 13800, loss: 104.884140
----
 ，笼罩在无数一浆羽的黑暗里，肯道在实音释空间模糊般了，漾眼里的簇一声，他像是吹望着结动着什么热纹树絮的地摆稍睡若来的控罩在风形，那群暴割着锁体中洒，那个人，围雪的黏蚀使者做。

    【魂雾流易的未层之止。

    “不对”三乎【雪暗】生着了，背后凝脱大留大睛，她心里刚以得边挟的脸在一边，越来及会有她的寒被，几乎好定的巨大猛着，笔带而大地的魂力的冰蔓，这里都游诲下来了那种人大自己的黑雪。

  
----
iter 13900, loss: 104.369215
----
 后来，一处一种反淌了，如颗钟数掠胡袍沉动着缠蝶的巨大的白色结形，一个箩现在高膛状喻，也和海呼的所陪，幽室般拍亮沿着气里，把地站在神音从来突然他转空间的悬落剔后世界的惨裂一起来，这个巨大的脸子。

    脖命像是他抬起亮此刻的尽头，子里的氤燥开去，然后的汩掠，一片，风影轻并身进称裸的一股双手角凸出卷动起来，突然去地更轻地上碎成着千体云啸建镜般，像是赫着的情失上来最后的动情把他的惨源回路。

    
----
iter 14000, loss: 104.780870
----
 ，卷着从开来开来般的水又力出始察聚红色，谁道么伊‘。

...

----
 地立，甚至还有把手流肩膀上闪电般的刀口，环缠绕起来一一恢复的世间里，对徊手就在小水划出手悚然，草丝。此刻。银尘一直看着整男子看着那四巨大的血部瞬间瞬间最无体内味的时指滑翔着说：“口都是又杀什么价料的称拉和是什么怪物吧…………要柴重了……您了鬼山缝魂住银尘眼前沉埃太清色的煦流一声被抵去的怒吼，在火风疯满无辜而挣。他转身的鼻屿回路当然日面跟出来。漆拉翻涌过来的一周震，周围从地把苍白的神志邪气朝下方点头 
----
iter 98700, loss: 84.020748
----
 力仿佛热骨海笼咆哮五空只亚斯蓝的天赋，在这个好害源而最重，这要大底是刚刚学下降的乎没有就告诉了什么为！”

    灵天十两碧绿大齐地，在自己顶上，她伸出树秒的石碑下顶了周围面的一光仿佛浓密麻麻芳沸的天空里，这无金色魂力深源，还是为什么操独过，你和我以发生命上的………到是生路不懂了什么主位之后的事生命令”，她一张大眼的花瓣，然感觉到莲泉的背后自己魂力的战个盒子，都再当我轻轻地说了， “你不好，我觉 
----
iter 98800, loss: 83.971287
----
 过眼他看起来，成了大洞的长袍，仿佛被风华的闇翅延眼的形块，“那么可以，我还是不清楚地对魂兽的骇名表情更加得带的一种魂器，“好我们在是水言啪留在像是一个巨大大海的灵魂回路一套的场景意思，麒零的视线依然当尘……这天束海自己想在不断的亡灵里。他很抱住骨着佩糙瞬间的空气里。

    

    

    “哦！都把伴并不知道们剩下僵硬的魂兽，充满了几个他赤裸的黑暗里，瞬间，他在他娇谲而卡的悲伤，“铜雀 
----
iter 98900, loss: 84.299196
----
   “天你！”鬼山莲泉像是鹿觉有麒零，开喉风中的方式被草上去。

    托卡也像是她白银祭司如同个盛离着城市浪─无数的甬道被宽恕铺着尤图尔开出的美丽回过头，仿佛是十万年外的光芒，漆拉涣得更刺汹涌了的庆吼，他贴着清晰的身躯缠绕在着格兰仕俯低一眼含白色。皎昏暗着一股躲转，“哦，我跑哪格外醒。天束幽花依然得张大惊的格石，仿佛天束幽花看着银白色的神音般喊着，但是，只体一个小的岁的样锋撞，银尘一下想答，他 
----
iter 99000, loss: 83.947473
----
 因为精致而没有事戮的事情，对我和囚禁吉尔伽美什，知道他正在我

----
 深处的向丝绸盆外笑意上，一点儿而邪涨地震，讥讽笑容在到底遥离一颗一样的黑色的岩石吹得如那就在北洋般的年前面前的拳头看起走去，既然的银尘对顶着空旷的女子的光滑熟暗地目光里缓慢地愈合，双钳无沫聚成无数尖游的石柱，划然临意而进入鬓站的手臂，仿佛被火石不胎儿了混浊，如同湖面之前的房间时这个穿时脚边并其因为他们自己亲眼的银尘，这片岛屿里此刻已经消失的海面上来的最壤高的魂力融化成一炉十十口和海域里逃脱，会特的 
----
iter 102100, loss: 82.987153
----
 心里和远处烤。

    “请自己走到。因为，认我们到这些灾受的胸口一点儿，因为她的手腕上那块散发着一雪温柔的吞噬，站起来，就当银白色一锤被一堆建筑里这个岛屿是，非常妩骸。

    “不是我们也缓慢侧回荡开来，庞大天束幽花点点儿自己们反而的表情之间，是截然也之情逃上……你是这里，逐渐撕开手，再过程度，我赶紧回来他又放开银身，巨大海面上想……如全一个它，怎么来。因为来一来自，甚至看见底发， “这种 
----
iter 102200, loss: 82.732391
----
 美，而且追趋正上怎么会知道，还有的触怪了……

    鬼里

    “个方，甚至是一手，收起类大型的将限度的天神的抗撼和抢，示女魂力他在在一种根隙里突悄拔醒地藏着尾风源，抓住一度王爵，尽面了你们，我的手上封印，他聪明到赋风类……或者操纵性定对，水边却至是很时间有廊中的魂力的那也不能没力，但这位回廊，天袍里突冒出模热祭司，他们所有人，被洞穴拼上树位的力量残。”草坪里进了个白眼神秘。”鬼山莲泉不让 
----
iter 102300, loss: 82.650779
----
 力元素时候，过了普重。

    面发出一个金色光闹的水，浑红的轮廓走廊上，通穴地源格断利的，连石棒空一样撕开她柔的曼椎佛飞落的气流扎，语山莲泉，转过头颅，仿佛隐隐开始的尖刀金红，在冷离而整个土翼，凝实在那么茫道降，  银尘完全一挥，没有因为，让要万丈了。麒零，从特蕾娅高地之前的眼，又自己的感开就不过来，轻男人看起来，周围空间的视影，在岸时，往格兰仕，三天空仿佛突然感应着那种黑蓝色床吹紧打丛里，她 
----
iter 102400, loss: 82.907445
----
 ……终于说，出现性的魂术编织包荡起，巨大的尽室气什么

----
 涌，而是艾尔致身完全的想得感觉得接一个人喃呐弓紧着茶晕如同一层悠昧的。

    莲泉的王爵一下，三个小女孩的风容无数一张的声音下，重新地告脸闭上眼睛，仿佛亚斯蓝帝国的群，正疮吗来吧，看见我来王爵了，退下如果正一碗一样猎福，成为她感觉行久，连我现在的眉眼，任何的帝都已儿和你这些东西之前等我一个一办，他感觉在他的心里，能够说要你，不，我应该才能就去，这里最高什么变故，哗啦啦地跃了吧地，我告诉了”魂兽 
----
iter 105500, loss: 84.230216
----
 则我才可能推……”身边抬起头，身体砸扬起来，声的火鸟延立的白色洁刺年密安静的笑容，却看非样的淡淡楚正你的声音注过三人，一个白银难个王爵“。作为我要完美，而且，我给了！”银尘像是大厚的声音是看了，周围的绿色砍镶般的血骸，又听渡问道，  “这是……听我只是在等什么？”

    银尘摇着叮泽，银尘地站起来没关离而触”赐印狭长的血肉。

    银尘的耳朵去垂直接在两人突然朝着站在瓦口，一边不过不了。前 
----
iter 105600, loss: 84.098104
----
 射出片海域上黑色的效下来，被害怕，提前两个神，和他心脏吹动着月光照见。

    神音依然是无数大海块时的头顶扶绑天空突然爆炸出如同深处一片，此刻也再要回答得更加失去。

    麒零目光里，一边缓慢地朝它滴答，我在好远的冥背，从眼睛仰想着他的眼神，终于是在一道瘆人的笑理那动声，眼神低铃脑夹过身后的是石沉重新攻击。

    致子看起实力，一碎一兽一散，一片琥珀里面的，变成一声准备“可以，正怕么… 
----
iter 105700, loss: 84.359679
----
 魂力已经像是深渊回廊镜面的人整个男孩子切啊，完全发出半三冰心脏里红色的寒冷，似乎比袍准提前问驿之外，曾经一个半楼划起，一个标涡。

    银尘地瞄着幽暗的表情不断地，应该是把麒零渐渐消失去。嘻

    “莲泉目熟长袍，然牢爆炸来的白色丝吸呢。，依然消失成两层锋利的鲜血笼烂狂爆炸，因为了拔小即缠立，来了勇气的一样。

    这种做动，加速闷温润了宝异，没有人跑起来，一起地字滑了两条和天赋更听， 
----
iter 105800, loss: 83.931448
----
 为一摊十米的风彼此在自己的血量用力，然后也更恢复往前

iter 108800, loss: 80.551077
----
 使徒”

    她张重地面容，看着漆拉轻整把黑色眼蟒。

    浓成长袍的瞬间，无刻不短的时，顷刻间有生见幽花和幽花只中的巨型的个手之下，周围丽长的山崖上，伴留在不知道空旷的白色。

    他了苍白了暗。

    漆拉三个说。他跑入桌子上把大古城散旋在旁边的鬼魅，看着没有生命了进路深渊回廊里的阵营，”幽冥跪在视线“妹描”尘扑着，“也不喵”的人问。

    麒零很清晰地望着对面，他一棵城线 
----
iter 108900, loss: 80.293756
----
 转了，脸音不微笑地笑，像是章   口的银尘。

    神秘密丝、麒零都像是从月色的空间里，那一瞬间飞禽散去，把两根昏厥的感应着，看起来没有看清晰地低头打出来了和霓雅，用力量地摇了摇头下来。

    【西属隙里处去如同”七度王爵“都，哦体内。”

    “嗒哈乐好！我，她了问题，又是也不刚刚和这处有一个日子。

    “我们是亚斯蓝国家，并不像是涨本不能挑底，天束幽花拿走到银关全六度王爵】的 
----
iter 109000, loss: 80.859046
----
 却像是在空气里比了几人，仿佛金斯的洞，吉尔伽美什要显得融合进来，它往的保护泥实是，对南万年的雾气里。”

    “哦，我现在使徒“其实不知道不过很久的【战赋】最称”瞬间，“幽冥吧？是不可能捡让人的问题平域上”火晶】飞去的店伤的大殿，看着想心里，松了。完全有一颗地图笑。

    帝国.己飞快而探】走过哪样。没有要这发生怪生，完散不断的迷乱中的宽阔，路过一个特集的关系通的智慧品物状态啊……”天束幽 
----
iter 109100, loss: 80.883751
----
    “不过世界之后，早是这么威存的那么能留下去之在大体里略就难道好，是真的二名么乱，就像在少”充满着艳丽眼睛里，射现出来的汁妒，一声怒气流扎黏忌祸的一种感觉。没儿得及笑，“再也觉得了东形容力而是……”

    “我站出来的鬼魅，除了一种动天地上。

    “欺联”七度王爵“谁？”

    “宏极历你，”幽花的银尘，眼前的束花面前的右花在空气上，滚发。麒零转过去，特神的挑风在没兽白，但是…… 
----
iter 109200, loss: 81.134579
---

----
 然空气，巨大的脸庞大盾牌们绵般的喉咙里扭曲声音，不慢地上面朝水往上海风的黑色冰风，切的锁眼，走到那条难暴刘地分娩出这种植伤害怕，而是一个少年了，像可如此同时，都再一度王爵，如果走吓人的事情？只是瞬间将漆黑的银尘，似乎结正的铜刷“使徒多么？眸子了此份呢。

    麒零抓起身间的胸玩平静，还去，他想到说里面向他们，于其实，如同从湖腹拥有动命的怪物么？

    银尘的脚下滑塑，海岸边的那个一点……着 
----
iter 112300, loss: 82.971952
----
 动着的莉吉尔苍白。

    准汰的边鬼的莲泉笑容进去。

   而是空中底量里来。但是，从自己看上纤细的晦纹一股来骇人望着血巅们洞穴干净的。麒零眯着面容，就复逗什么，其实看见特蕾娅白色的魂力边缘，然后的苍白的眉毛，仿佛在雪气的场景在从眼睛分崖上，也突然在她心里又又像一套人离开了么？！特蕾娅的方音，带在她脸上两眼赐印。她抬起嘴里，她将靠近云焦的大光填满，胧讶地说不惜，走道，也是一直去魂术，不能自隐 
----
iter 112400, loss: 83.196188
----
 的东西。所以，她也到对面前方之后，此刻就是谁好说过来的是囚禁过的王爵遇到这座大海风里在你布里比嗖于回生的毫亲人都就正在刚杀了。我现在那么不奇来回答。”天束幽花，那天空中被一片雪气里太过去，是它们场褴颖年轻轻地笑，“我来说，不至任何成雷恩的裙境里的衣服。

    但而又步口气头。麒零望着看清楚。然而在神开了。她的边卫里面有一句银尘的脸上。

    “她们正轻地共位的使者一颗一种独到的人知道，风源 
----
iter 112500, loss: 83.048120
----
 中的瞬间，这片丝毫不会知道们的女前场景，看见吉尔伽美什身体像是轻地跌了气了，然后，仿佛从最高的眸子里同样去依然看着双臂爬下坠落放寂来。莲泉眼，面前躺在阴森的视怖岛屿。无数在泉身边寻回路巨大的距心里去，他却看不去无数湛细细长的脚步里浮现出的重意说，吓得人。

    巨浪般闪动的水猛突然冷声像是只剩下男孩都觉得不堪，她的魂兽，协进霓时，放在这些轰然袖子：声音也充满了怕修长啸的眼睑上，她虽然知道跟了摇 
----
iter 112600, loss: 82.775607
----
 银尘。

    特蕾娅回头感离摇头，天束幽花突然出